# Датасет

Я решил использовать открытый датасет при использовании базовых утверждений

In [6]:
!wget https://huggingface.co/api/datasets/deepvk/ru-HNP/parquet/default/train/0.parquet

--2025-01-08 22:41:19--  https://huggingface.co/api/datasets/deepvk/ru-HNP/parquet/default/train/0.parquet
Resolving huggingface.co (huggingface.co)... 18.172.134.4, 18.172.134.88, 18.172.134.124, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://huggingface.co/datasets/deepvk/ru-HNP/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet [following]
--2025-01-08 22:41:19--  https://huggingface.co/datasets/deepvk/ru-HNP/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/f6/05/f6058a3ecc13aab343cd0789dd84886ca237aa7b1c2c33817ccb0b15b7635368/210c0a80185c430aa0e2d62dff9e56ca3a142919d064582161d019069e769a7e?response-content-disposition=inline%3B+filename*%3DUTF-8%27%270000.parquet%3B+filename%3D%220000.parquet%22%3B&Expires=1736635279&Po

In [35]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

In [85]:
df = pd.read_parquet("0.parquet")[["query"]][:100]

In [86]:
df

,query
0,С 1952 года после окончания Куйбышевского инду...
1,Впадающие реки: Гаронна и Дордонь (образующие ...
2,После смерти отца в 1709 году Людовик унаследо...
3,С началом Второй мировой войны клиника перешла...
4,"Родники являются местом паломничества, так как..."
...,...
95,В июне 2012 года депутаты Заксобрания Забайкал...
96,Проблемы токсикологии продуктов питания и гиги...
97,Спортивное общество «Калев» () — эстонское спо...
98,В зависимости от модификации мины взрыв мог ра...


## GPT2 модель генерации

In [91]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')

gpt_phrases = []

for text in tqdm(df["query"].values):
    gpt_phrases.append(generator(text, max_length=int(len(text) * 1.5 + 10), num_return_sequences=1, truncation=True))

Device set to use cpu


  0%|          | 0/100 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [98]:
df["gpt"] = gpt_phrases
df["gpt"] = df["gpt"].progress_apply(lambda x: x[0]['generated_text'])

  0%|          | 0/100 [00:00<?, ?it/s]

## ruT5 модель генерации

In [87]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

MODEL_NAME = 'cointegrated/rut5-base-paraphraser'

model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

In [88]:
# model.cuda()
model.eval()

def paraphrase(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=True) # , do_sample=do_sample
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [89]:
phrases = []

for text in tqdm(df["query"].values):
    phrases.append(paraphrase(text, 5, 4))

  0%|          | 0/100 [00:00<?, ?it/s]

In [100]:
df["rut"] = phrases

## mT5 модель генерации (Не получилось запустить, не билдится wheel)

In [116]:
# !pip install russian_paraphrasers
# !pip install gensim
# !pip install git+https://github.com/Maluuba/nlg-eval.git@master

  Cloning https://github.com/Maluuba/nlg-eval.git (to revision master) to /tmp/pip-req-build-qxn777fs
  Running command git clone --filter=blob:none --quiet https://github.com/Maluuba/nlg-eval.git /tmp/pip-req-build-qxn777fs
  Resolved https://github.com/Maluuba/nlg-eval.git to commit 2ab4528fad5548315cf61e40c2249fec8c8ad233
  Preparing metadata (setup.py) ... done
  Using cached gensim-3.8.3.tar.gz (23.4 MB)
  Preparing metadata (setup.py) ... done
  Using cached Theano-1.0.5-py3-none-any.whl
  Using cached xdg-6.0.0-py3-none-any.whl.metadata (1.3 kB)
Using cached xdg-6.0.0-py3-none-any.whl (3.9 kB)
  Created wheel for nlg-eval: filename=nlg_eval-2.4.1-py3-none-any.whl size=98924372 sha256=5a69ecec600733ec6e69ba736583d2bcaf3867614ae819c8f88f9390fd05997e
  Stored in directory: /tmp/pip-ephem-wheel-cache-5cgguqcg/wheels/f8/9e/cd/603676036c65a4c71210f6e96dad569e3a9d4a3448acf40798
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit c

In [112]:
# from russian_paraphrasers import Mt5Paraphraser

# paraphraser = Mt5Paraphraser(model_name="mt5-base", range_cand=False, make_eval=False)

ModuleNotFoundError: No module named 'nlgeval'

In [ ]:
# mt_phrases = []

# for text in tqdm(df["query"].values):
#     phrases.append(
#         paraphraser.generate(
#             text, n=1, temperature=1,
#             top_k=10, top_p=0.9,
#             max_length=int(len(text) * 1.5 + 10), repetition_penalty=1.5,
#             threshold=0.7
#         )
#     )

In [ ]:
# df["mt"] = mt_phrases

# Валидация результатов

Интересно было сравнить результаты двух рефразийных моделей  
Посчитал метрики на основе исходных утверждений и сгенерированных двумя моделями  
- **Cosine Similarity** на эмбеддингах русскоязычного BeRT как метрику семантической близости
- **BLEU** как метрику доли перефразирования

## Расчет семантической близости

In [121]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")


def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

query_embeddings = []
for text in tqdm(df["query"].values):
    query_embeddings.append(embed_bert_cls(text, model, tokenizer))
df["query_embeddings"] = query_embeddings

gpt_embeddings = []
for text in tqdm(df["gpt"].values):
    gpt_embeddings.append(embed_bert_cls(text, model, tokenizer))
df["gpt_embeddings"] = gpt_embeddings

rut_embeddings = []
for text in tqdm(df["rut"].values):
    rut_embeddings.append(embed_bert_cls(text, model, tokenizer))
df["rut_embeddings"] = rut_embeddings

# mt_embeddings = []
# for text in tqdm(df["mt"].values):
#     phrases.append(embed_bert_cls(text, model, tokenizer))
# df["mt_embeddings"] = mt_embeddings

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [176]:
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-08)

df["gpt_cossim"] = df.progress_apply(lambda x: cos(torch.Tensor(x["query_embeddings"]), torch.Tensor(x["gpt_embeddings"])).item(), axis=1)
df["rut_cossim"] = df.progress_apply(lambda x: cos(torch.Tensor(x["query_embeddings"]), torch.Tensor(x["rut_embeddings"])).item(), axis=1)
# df["mt_cossim"] = df.progress_apply(lambda x: cos(torch.Tensor(x["query_embeddings"]), torch.Tensor(x["mt_embeddings"])), axis=1)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

## Расчет доли повторения

In [180]:
from nltk.translate.bleu_score import sentence_bleu

In [182]:
df["gpt_bleu"] = df.progress_apply(lambda x: sentence_bleu([x["query"].split()], x["gpt"].split()) * 100, axis=1)
df["rut_bleu"] = df.progress_apply(lambda x: sentence_bleu([x["query"].split()], x["rut"].split()) * 100, axis=1)
# df["mt_bleu"] = df.progress_apply(lambda x: sentence_bleu([x["query"].str.split()], x["mt"].str.split()) * 100, axis=1)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [188]:
pd.DataFrame({"Cosine Similarity": {"GPT2": df["gpt_cossim"].mean(),
                                    "ruT5": df["rut_cossim"].mean()},
              "BLEU (%)": {"GPT2": df["gpt_bleu"].mean(),
                       "ruT5": df["rut_bleu"].mean()}})

,Cosine Similarity,BLEU (%)
GPT2,0.915238,55.318767
ruT5,0.955238,1.033532


In [196]:
from tabulate import tabulate

print(tabulate(df[["query", "gpt", "rut"]].T, headers='keys', tablefmt='psql'))

+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------